# HW1 - Intro to the Map Reduce Paradigm  
__ `MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Spring 2018`__

Welcome to Machine Learning at Scale! This first homework assignment introduces one of the core strategies in distributed processing: divide and conquer. We'll use the simplest of tasks, word counting, to illustrate the difference between a scalable and non-scalable algorithm. You will be working with the text of _Alice in Wonderland_ to put these ideas into practice using Python and Bash scripting. By the end of this week you should be able to:
* ... __describe__ the Bias-Variance tradeoff as it applies to Machine Learning.
* ... __explain__ why we consider word counting to be an "Embarrassingly Parallel" task.
* ... __estimate__ the runtime of embarrassingly parallel tasks using "back of the envelope" calculations.
* ... __implement__ a Map Reduce algorithm using the Command Line.
* ... __set-up__ a Docker container and know why we use them for this course.

You will also  become familiar (if you aren't already) with `defaultdict`, `re` and `time` in Python, linux piping and sorting, and Jupyter magic commands `%%writefile` and `%%timeit`. __Please refer to the `README` for detailed submission instructions__.

__IMPORTANT:__ If you're not familiar with linux, you should read the following tutorial reagrding **piping** and **redirecting**: https://ryanstutorials.net/linuxtutorial/piping.php You will need to understand the differences to answer some of the later questions.


# Notebook Set-Up
Before starting your homework run the following cells to confirm your setup.

In [1]:
# confirm you are running Python 3
import sys
sys.version_info

sys.version_info(major=3, minor=7, micro=5, releaselevel='final', serial=0)

In [2]:
# imports
import re
import sys

Create a folder for any data you download locally.

In [77]:
!mkdir data
# NOTE: the contents of this directory will be ignored by git.

mkdir: cannot create directory `data': File exists


# Question 1: Introductions

`The Caterpillar and Alice looked at each other for some time in silence: at last the Caterpillar took the hookah out of its mouth, and addressed her in a languid, sleepy voice. "Who are you?" said the Caterpillar.`   
<div style="text-align: right"> -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4 </div>


__Tell us about yourself! Briefly describe where you live, how far along you are in MIDS, what other classes you are taking and what you want to get out of w261.__

> Hello, my name is Samuel Gomez.  I live in California's Central Valley.  I grew up on a 100-acre tree fruit farm and competed in wrestling when I was younger.  After high school, I majored in electrical engineering focused on automation (control systems and robotics).  For the last seven years, I have worked as the Director of Engineering for Tasteful Selections--the largest bite-size baby potato distributor in the U.S.  I was part of the original startup team and supported engineering, information technology, project management, and, most recently, data analytics.  I grew these departments from one to twenty-five while supporting tens of millions in annual CapEx projects and day-to-day operations.  In our first year of business, we set a record 300,000lbs shipped in one week; during my last year with the company, we shipped 6,300,000lbs in one week.  

>I recently transitioned to my second startup company, Concentric Power.  We design, build, install, and commission microgrids.  As the Director of Microgrid Intelligence and Automation, I oversee the automation and controls, information technology, and data science departments.  Our mission is to revolutionize the country's energy grid through new technologies.

>W261 is my only class this semester.  It is my last elective; I plan to enroll in capstone next semester. I want to learn more about the framework necessary to analyze data at scale.  I also want to gain experience in deploying algorithms other than deep neural nets on large datasets.

# Question 2: Bias - Variance
__In 1-2 sentences explain the bias-variance trade off. Describe what it means to "decompose" sources of error. How is this used in machine learning?__ Please use mathematical equation(s) to support your explanation. (Use `$` signs to take advantage of $\LaTeX$ formatting, eg. `$f(x)$` will look like: $f(x)$). Please also cite any sources that informed your answer.

> The bias-variance tradeoff is the property of a model that the variance of the parameter estimated across samples can be reduced by increasing the bias in the estimated parameters. Models with high bias tend to be simple and could miss the relevant relationships between features and target outputs.  On the other hand, models with high variance tend to be complex and can overfit the training data resulting in poor generalization.   It is important to note that the objective of any model is to reduce the overall error.  That said, the sweet spot for any model is achieving a level of complexity at which an increase in bias is equivalent to the reduction in variance (An inverse relationship exists between bias and variance.)

> Reducible and irreducible errors make up a model's overall error. Irreducible error or inherent uncertainty is associated with natural variability in a system (e.g., noisy sensors, incorrectly documented data, etc.). On the other hand, as the name suggests, the reducible error can and should be minimized to maximize a model's accuracy.  The reducible error includes errors due to squared bias, variance, and noise.  
> Reducible Error = $E = bias^2 + var + noise$

# Question 3: Tokenizing
A number of our assignments this term will involve extracting information from text. A common preprocessing step when working with raw files is to 'tokenize' (i.e. extract words from) the text. Within the field of Natural Language Processing a lot of thought goes into what specific tokenizing makes most sense for a given task. For example, you might choose to remove punctuation or to consider punctuation symbols  'tokens' in their own right. __In this question you'll use the Python `re` module to create a tokenizer to use when you perform WordCount on the _Alice In Wonderland_ text.__

### Q3 Tasks:

* __a) short response:__ In the Naive Bayes algorithm (which we'll implement next week), we'll estimate the _likelihood_ of a word by counting the number of times it appears and dividing by the size of the vocabulary (total number of unique words). Using the text: *"Alice had an adventure that took alice to wonderland"*, give a concrete example of how two different tokenizers could cause us to get two different results on this calculation. [`HINT`: _you should not need to read up on Naive Bayes to answer this question_]  
  

* __b) short response:__ When tokenizing in this assignment we'll remove punctuation and discard numerical digits by making everything lowercase and then capturing only consecutive letters a to z. Suppose __`tokenize(x)`__ is a Python function that performs the desired tokenization. What would __`tokenize("By-the-bye, what became of Alice's 12 hats?!")`__ output? Type the answer in the space provided below.


* __c) code:__  Fill in the regular expression pattern in the cell labeled `part c` so that the subsequent call to `re.findall(RE_PATTERN, ...)` returns the tokenization described above. [`HINT`: _we've taken care of the lowercase part for you. If regex is new to you, check out the [`re`  documentation](https://docs.python.org/3/library/re.html) or [this PyMOTW tutorial](https://pymotw.com/2/re/)._]

### Q3 Student Answers:
> __a)__ In a situation where one tokenizer implements a lower() method and one does not, the outcome differs with one list containing ['alice'] and the other containing ['Alice', 'alice'] respectively.  Thus, this is a concrete example of how two tokenizer's calculations can differ. 

> __b)__ Considering punctuation is removed, tokenize(x) will output the following:  
> ['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']

In [13]:
# Part C - Fill in the regular expression
RE_PATTERN = re.compile("[a-z]+")          

In [14]:
# Tokenize - DO NOT MODIFY THIS CELL, just run it as is to check your pattern
words = re.findall(RE_PATTERN, "By-the-bye, what became of Alice's 12 hats?!".lower())
print(words)

['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']


# Load the Data
` "Please would you tell me", said Alice, a little timidly, for she was not quite sure whether it was good manners for her to speak first, "why your cat grins like that?"`  
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4  </div>

For the main part of this assignment we'll be working with the free plain text version of _Alice's Adventures in Wonderland_ available from Project Gutenberg. __Use the first two cells below to download this text and preview the first few lines.__ 

In [80]:
# Download Full text 
# NOTE: feel free to replace 'curl' with 'wget' or equivalent command of your choice.
!wget "http://www.gutenberg.org/files/11/11-0.txt" -o data/alice.txt

In [3]:
# Take a peak at the first few lines
!head -n 6 data/alice.txt
# NOTE: If you are working in JupyterLab on Docker you may not see the output 
# below due to an encoding issue... in that case, use a terminal on Docker to 
# execute this head command and confirm that the file has downloaded properly, 
# this encoding issue should not affect your work on subsequent HW items.

﻿The Project Gutenberg eBook of Alice’s Adventures in Wonderland, by Lewis Carroll

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at


We'd also like you to develop a habit of creating small files with simulated data for use in developing, debugging and testing your code. The jupyter magic command `%%writefile` is a convenient way to do this. __Run the following cells to create a test data file for use in our word counting task.__

In [83]:
%%writefile data/alice_test.txt
This is a small test file. This file is for a test.
This small test file has two small lines.

Writing data/alice_test.txt


In [84]:
# confirm the file was created in the data directory using a grep command:
!ls data | grep test

alice_test.txt


In [85]:
# testing cat command
! cat data/alice_test.txt

This is a small test file. This file is for a test.
This small test file has two small lines.


# Question 4: Word Count in Python

Over the course of this term you will also become very familiar writing Python programs that read from standard input and using Linux piping commands to run these programs and save their output to file. __In this question you will write a short python script to perform the Word Count task and then run your script on the _Alice in Wonderland_ text__. You can think of this like a 'baseline implementation' that we'll later compare to the parallelized version of the same task.

### Q4 Tasks:

* __a) code:__ Complete the Python script in the file __`wordCount.py`__. Read the docstrings carefully to be sure you understand the expected behavior of this function. Please do not code outside of the marked location.


* __b) testing:__ Run the cell marked `part b` to call your script on the test file we created above. Confirm that your script returns the correct counts for each word by visually comparing the output to the test file. 


* __c) results:__ When you are confident in your implementation, run the cell marked `part c` to count the number of occurrences of each word in _Alice's Adventures in Wonderland_. In the same cell we'll pipe the output to file. Then use the provided `grep` commands to check your answers.


* __d) short response:__ Suppose you decide that you'd really like  a word and its plural (e.g. 'hatter' and 'hatters' or 'person' and 'people') to be counted as the same word. After we have run the wordcount would it be more efficient to post-process your output file or discard your output file and start the analysis over with a new tokenizer? Explain your reasoning briefly. 

### Q4 Student Answers:
> __a-c)__ _Complete the coding portions of this question before answering 'e'._

> __d)__ Post-processing the output file is typically more efficient in this scenario.  In most implementations, the output file contains one key-value pair for each distinct token, while the corpus typically contains far more data.  Therefore, the output file requires less computation compared to the corpus.  The corpus size ultimately determines the best approach for the desired outcome.  Although both the post-processing and re-running the anlaysis approaches may have similar computation time if the corpus is relatively small.  However, in an application at scale where big data has undergone analysis, it's more efficient to post-process.  The point is this; as an algorithm scales, the reduced output file pales in comparison to the size of the input files.

In [86]:
# part a - DO YOUR WORK IN wordCount.py

In [87]:
!cat data/alice_test.txt

This is a small test file. This file is for a test.
This small test file has two small lines.


In [88]:
# part b - DO NOT MODIFY THIS CELL, just run it as is to test your script
!python wordCount.py < data/alice_test.txt

this 	 3
is 	 2
a 	 2
small 	 3
test 	 3
file 	 3
for 	 1
has 	 1
two 	 1
lines 	 1


In [89]:
# part c - DO NOT MODIFY THIS CELL, just run it as is to perform the word count.
!python wordCount.py < data/alice.txt > data/alice_counts.txt

Take a look at the first 10 words & their counts.

In [90]:
!head data/alice_counts.txt

the 	 1839
project 	 88
gutenberg 	 98
ebook 	 13
of 	 638
alice 	 403
s 	 222
adventures 	 11
in 	 435
wonderland 	 7


__Check your results:__ How many times does the word "alice" appear in the book? 

In [91]:
# EXPECTED OUTPUT: 404
!grep alice data/alice_counts.txt

alice 	 403


__Check your results:__ How many times does the word "hatter" appear in the book? 

In [92]:
# EXPECTED OUTPUT: 56
!grep hatter data/alice_counts.txt

hatter 	 56
hatters 	 1


__Check your results:__ How many times does the word "queen" appear in the book? 

In [93]:
# EXPECTED OUTPUT: 76
!grep queen data/alice_counts.txt

queen 	 76
queens 	 1


# Question 5: Unix Sorting Practice
Another common task in this course's assignments will be to make strategic use of sorting.     

### Q5 Tasks:
* __a) short response:__ What is the Big O complexity of the fastest comparison based sorting algorithms? [*HINT: If you need a Big O notation refresher, here's_ a [blog post](https://rob-bell.net/2009/06/a-beginners-guide-to-big-o-notation/), a [cheatsheet](http://bigocheatsheet.com), and a [thorough explanation](http://pages.cs.wisc.edu/~vernon/cs367/notes/3.COMPLEXITY.html).*]

* __b) short response:__ What is the default sorting algorithm in MapReduce? What is the Big O complexity of this algorithm? Why do you think this algorithm was chosen? [*HINT: Julius Ceasar! (week 1 slides)*]

* __c) code:__ write a unix command to check how many records are in your word count file.

* __d) code:__ Write a unix command to sort your word count file alphabetically. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) the results to `data/alice_counts_A-Z.txt`. [*HINT: if Unix sort commands are new to you, start with [this biowize blogpost](https://biowize.wordpress.com/2015/03/13/unix-sort-sorting-with-both-numeric-and-non-numeric-keys/) or [this unixschool tutorial](http://www.theunixschool.com/2012/08/linux-sort-command-examples.html)*]

* __e) code:__ Write a unix command to sort your word count file from highest to lowest count. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) your results to `data/alice_counts_sorted.txt`; then run the provided cell to print the top ten words. Compare your output to the expected output we provide.

### Q5 Student Answers:
> __a)__ The Big O complexity of the fastest comparison-based sorting algorithm is $O(n)$ at best.  Keep in mind that this performance occurs **at best** and is highly unlikely in real-world applications. For example, a bubble sort can perform $O(n)$ if it sorts a list in only one pass.  Otherwise, it performs $O(n^2)$, which is not desirable and occurs more often than not.  A comparison-based sorting algorithm such as mergesort, which is an efficient, general-purpose, and widely used sorting method, typically performs $O$ $(n$ $log$ $n)$.  Taking into consideration that algorithms will scale to big data, the fastest computation is regularly $O$ $(n$ $log$ $n)$

> __b)__ The default sorting algorithm in MapReduce is mergesort.  The Big O complexity of mergesort is $O$ $(n$ $log$ $n)$ for best, average, and worst-case performance.  This algorithm is a good choice for MapReduce because, at worst, it still performs well.  Most implementations produce a stable sort--meaning whenever there are two records R and S with the same key and R appears before S in the original list, R will appear before S in the sorted list.  Another benefit of mergesort is the ability to divide and conquer--meaning a mergesort job can run on multiple hardware in parallel for faster computation. 

In [94]:
# part c - write a unix command to check how many records are in your word count file
!wc -l data/alice_counts.txt

3006 data/alice_counts.txt


In [95]:
# part d - unix command to sort your word counts alphabetically 
!sort data/alice_counts.txt > data/alice_counts_A-Z.txt

In [96]:
# part d - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_A-Z.txt   

a 	 695
abide 	 2
able 	 1
about 	 102
above 	 3
absence 	 1
absurd 	 2
accept 	 1
acceptance 	 1
accepted 	 2


In [133]:
# part e - unix command to sort your word counts from highest to lowest count
!sort -nr -k2 data/alice_counts.txt > data/alice_counts_sorted.txt

In [134]:
# part e - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_sorted.txt  

the 	 1839
and 	 942
to 	 811
a 	 695
of 	 638
it 	 610
she 	 553
i 	 546
you 	 486
said 	 462


<table>
<th>expected output for (d):</th>
<th>expected output for (e):</th>
<tr><td><pre>
a	695
abide	2
able	1
about	102
above	3
absence	1
absurd	2
accept	1
acceptance	1
accepted	2
</pre></td>
<td><pre>
the	1837
and	946
to	811
a	695
of	637
it	610
she	553
i	546
you	487
said	462
</pre></td></tr>
</table>

# Question 6: Parallel Word Count (part 1)
What would happen if we tried to run our script on a much larger dataset? For one thing, it would take longer to run. However there is a second concern. The Python object that aggregates our counts (`defaultdict`) exists in memory on the machine running this notebook. If the vocabulary is too large for the memory space available we would crash the notebook. The solution? Divide and Conquer! Instead of running the script on the whole dataset at once, we could split our text up in to smaller 'chunks' and process them independently of each other. __In this question you'll use a bash script to "parallelize" your Word Count.__


### Q6 Tasks:
* __a) read provided code:__ The bash script `pWordCount_v1.sh` takes an input file, splits it into a specified number of 'chunks', and then applies an executable of your choice to each chunk. Read through this code and make sure you understand each line before you proceed. [`HINT:` _For now, ignore the 'student code' section -- you'll use that in part c._]


* __b) short response:__ Below we've provided the command to use this script to apply your analysis (`wordCount.py`) to the _Alice_ text in 4 parallel processes. We'll redirect the results into a file called `alice_pCounts.txt.` Run this analysis and compare the count for the word 'alice' to your answer from Question 4. Explain what went wrong and describe what we have to add to `pWordCount_v1.sh` to fix the problem.


* __c) code:__ We've provided a python script, `aggregateCounts_v1.py`, which reads word counts from standard input and combines any duplicates it encounters. Read through this script to be sure you understand how it is written. Then follow the instructions in `pWordCount_v1.sh` to make a one-line modification so that it accepts `aggregateCounts_v1.py` as a 4th argument and uses this script to combine the chunk-ed word counts. Run the cell below to confirm that you now get the correct results for your 'alice' count.

### Q6 Student Answers:
> __b)__ The final output file is a result of concatenating the four chunked output files.  Notice that there are four lines of Alice, each with a respective value.  Adding up each value produces the total count calculated in the prior exercise.  In order to arrive at the desired outcome of one key-value pair in the output file, a reducer must be specified.


In [99]:
# part b - make sure your scripts are executable (RUN THIS CELL AS IS)
!chmod a+x pWordCount_v1.sh
!chmod a+x wordCount.py

In [100]:
# part b - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' 'wordCount.py' > 'data/alice_pCounts.txt'

In [101]:
# part b - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice 	 113
alice 	 126
alice 	 122
alice 	 42


In [102]:
# part c - make sure the aggregateCounts script is executable (RUN THIS CELL AS IS)
!chmod a+x aggregateCounts_v1.py

In [103]:
# part c - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v1.py' \
                   > 'data/alice_pCounts.txt'

In [104]:
# part c - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	403


In [105]:
!man sort

No manual entry for sort


# Question 7: Parallel Word Count (part 2)
Congratulations, you've just implemented a Map-Reduce algorithm! From here on out, we'll refer to the two python scripts you passed to `pWordCount_v1.sh` as '_mapper_' and '_reducer_'. The bash script itself served as our '_framework_' -- it split up the original input, then ___mapped___ our word counting script on to each chunk, then ___aggregated (a.k.a. reduced)___ the resulting files by piping them into our collation script.  Unfortunately, as you may have realized already, there is a major scalability concern with this particular implementation. __In this question you'll fix our implementation of parallel word count so that it will be scalable.__

__HINT:__ MapReduce uses the Merge-Sort algorithm under the hood. Linux `sort` command has a merge option which you can use to simiulate the MapReduce framework. Use the `man sort` command to find more information on this option. 

### Q7 Tasks:

* __a) short response:__ What is the potential scalability problem with the provided implementation of `aggregateCounts_v1.py`? Explain why this supposedly 'parallelized' Word Count wouldn't work on a really large input corpus.  [`HINT:` _see the intro to Q6_]


* __b) code:__ Fortunately, a 'strategic sort' can solve this problem. Read the instructions at the top of `pWordCount_v2.sh` carefully then make your changes that alphabetically sort the output from the mappers (`countfiles`) before piping them into the reducer script.


* __c) code:__ Write the main part of `aggregateCounts_v2.py` so that it takes advantage of the sorted input to add duplicate counts without storing the whole vocabulary in memory. Refer to the file docstring for more detailed instructions. Confirm that your implementation works by running it on both the test and true data files.


* __d) short response:__ If you are paying close attention, this rewritten reducer sets us up for a truly scalable solution, but doesn't get us all the way there. In particular, while we chunked our data so it can be processed by multiple mappers, we're still streaming the entire dataset through one reduce script. If the vocabulary is too large to fit on a single computer, we might split the word counts in half after sorting them, then perform the reducing on two separate machines. Explain what could go wrong with this approach. (For now, ignore the question of how we'd sort a dataset that is too large to fit on a single machine and just focus on what might be wrong about the result of this split-in-half reducing).


* __e) short response:__ Can you come up with a different way of splitting up the data that would allow us to perform the reducing on separate machines without needing any postprocessing? This is a theoretical question -- don't worry if you don't know how to implement your idea in a bash script, just describe how you'd want to split the sorted counts into different files to be reduced separately.

### Q7 Student Answers:

> __a)__ Two problems are present in version one of the aggregate counts implementation:  
> 1. No code is present to sort the count files; therefore, data can not be shuffled as key-value pairs to multiple reducers.  The inability to shuffle key-value pairs is a missed opportunity for taking advantage of parallel computing framework.  
> 2. Since a parallelized approach is off the table, one memory location must aggregate all the data.  An issue arises when a single machine's memory is not sufficient to accommodate the size of a dictionary produced from a large enough corpus.

> __b-c)__ _complete the coding portions of this question before answering d and e._

> __d)__ This approach contains no foolproof specification of splitting the file.  There is no guarantee the same key won't be sent to both reducers.  For example, it's possible to separate the file contents such that each reducer receives the same word (e.g., mad:1 + mad:1 = mad:2 sent to reducer-one and mad:1 = mad:1 sent to reducer-two).  Now that the same word exists in both reducers, the combined output file will contain two counts for "mad."

> __e)__ Implementing a method to seprate the file at an index where the current word and next word are different is one way to eliminate this problem.  Imagine we have a file that contains the following:  
a 173  
a 155  
a 201  
a 166  
abide 1  
abide 1   
able 1  
about 41  
about 36  
about 25  
In this example, splitting the contents between abide and able solves the issue. 

In [153]:
# Add permissions to your new files (RUN THIS CELL AS IS)
!chmod a+x pWordCount_v2.sh
!chmod a+x aggregateCounts_v2.py

In [154]:
# part c - test your code on the test file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice_test.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py'

a	2
file	3
for	1
has	1
is	2
lines	1
small	3
test	3
this	3


In [155]:
# part c - run your code on the Alice file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py' \
                   > 'data/alice_pCounts.txt'

In [156]:
# part c - confirm that your 'alice' count is correct (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	403


# Question 8: Scalability Considerations
In your reading for Week 2's live session, [Chapter1, section 2](https://lintool.github.io/MapReduceAlgorithms/MapReduce-book-final.pdf) of _Data Intensive Text Processing with MapReduce_, Lin and Dyer discuss a number of "Big Ideas" that underlie large scale processing: __scale "out," not "up"; assume failures are common; move processing to the data; process data sequentially and avoid random access; hide system-level details from the application developer; and seamless scalability.__ Part of our work this semester will be to interact with these ideas in a practical way, not just a conceptual one.

### Q8 Tasks:
* __a) short response:__ What do Lin and Dyer consider the two features of an "ideal algorithm" from a scalability perspective?


* __b) short response:__ The mapper script below (created on the fly using a little Jupyter magic) will help us illustrate the concept of scalability. Run the provided code which passes this mapper script to our parallel computation 'framework' and runs the 'analysis' on the _Alice_ text file. Note that we've omitted a reducer for simplicity. What do you observe about the time it takes to run this "algorithm" when we use 1, 2 and 4 partitions? Does it meet Lin and Dyer's criteria?


* __c) short response:__ Let's try something similar with your Word Count analysis. Run the provided code to time your implementation with 1, 2, 4 and 8 partitions. What do you observe about the runtimes? Does this match your expectation? Speculate about why we might be seeing these times. What conclusions should we draw about the scalability of our implementation? [`HINT:` _consider the limitations of both your machine and our implementation... there are some competing forces at work, what are they?_]


* __d) OPTIONAL:__ Which components of your Map-Reduce algorithm are affected by a change in the number of partitions? Does increasing the number of partitions increase or decrease the total time spent on each of these portions of the task? What tradeoff does this cause?

### Q8 Student Answers:

> __a)__ From a scalability perspective, Lin and Dyer consider the ideal algorithm to possess the following characteristics:  
        > 1. In terms of data: given twice the amount of data, the same algorithm should take at most twice as long to run, all else being equal.  
        > 2. In terms of resources: given a cluster twice the size, the same algorithm should take no more than half as long to run.

> __b)__ The algorithm's run time nearly decreased by the same factor the partitions increased, from one to two then four, respectively.    It seems debatable that Lin and Dyer's criteria apply to this implementation.  The data stays the same, ruling out criteria one.  Regarding criteria two, merely partitioning the data across CPUs is not analogous to increasing the cluster size.  
  
> Assuming criteria two is satisfied, and strictly the means (disregarding the std. devs.) are considered, the results do not meet Lin and Dyer's requirements.  The decrease in computing time for two and four partitions is more than 1/2 and 1/4 the compute time of one partition, respectively.  Recall that Lin and Dryer require the algorithm to take no more than half as long to run, given the cluster size doubles.    
  
| Partitions     | Mean (s) | Std. Dev (ms)     |
| :---:        |    :----:   |          :---: |
| 1     | 4.38    | 23.9 |
| 2     | 2.3     | 18 |
| 4     | 1.26    | 3.66 |

> __c)__ The run times are trending up as the number of partitions increase.  At first glance, these results seem counterintuitive.  However, the outcome is predictable when considering the mechanics of the implementation--let's take a look.  The step-by-step sequence and respective method of computation are as follows:
> 1. Partition - sequential computing.  This step is a sunk cost and can not improve as it relates to run time.
> 2. Mapper - parallel computing.  More resources will decrease run time.
> 3. Sort - sequential computing.  Opportunities such as putting the sort into the mapper provide parallel capabilities, thereby reducing run time.
> 4. Merge - sequential computing.  Cost of communication time due to networking increases with more partitions.  
> 5. Reduce - sequential computing.  Cost of communication time due to networking increases with more partitions.  More reducers should improve run time.

> Understanding the framework is crucial to understanding why the overall run time increases as the number of partitions increase.  During the sort, merge, and reduce steps, the communication cost amongst these processes increases as the number of partitions increaese due to their sequential computing nature.  With more partitions, more data must be moved around--this takes time and explains the upward time to compute trend we observed.  This exercise points out the balance between parallelizing tasks and communication time across the network while scaling up.  Also, keep in mind that this cloud instance has four CPUs, thereby limiting the benefits of more partitions to four.

| Partitions     | Mean (ms) | Std. Dev (ms)     |
| :---:        |    :----:   |          :---: |
| 1     | 269     | 4.71 |
| 2     | 272     | 8.35 |
| 4     | 271     | 4.15 |
| 8     | 309     | 4.06 |
| 16    | 379     | 7.24 |
| 32    | 523     | 4.15 |

> __d)__ Changing the number of partitions affects the process's chunking, mapping, sorting, merging, and reducing steps.  Note that moving sorting to the mapper creates a parallel computation for both the mapping and sorting steps, an advantage not yet utilized by this framework. As a rule of thumb, increasing partitions increases the time needed for sequential computations.  In this case, the time to partition, sort, merge, and reduce are all sequential processes; therefore, the time to complete their respective functions increases.  Mapping is the one step that benefits from increasing partitions but only to a certain extent.  The limiting factor corresponds to the number of CPUs on board the machine. 

__Run the following cells to create the mapper referenced in `part b`__

In [41]:
!mkdir demo

mkdir: cannot create directory `demo': File exists


In [157]:
%%writefile demo/mapper.py
#!/usr/bin/env python
"""
This mapper reads from STDIN and waits 0.001 seconds per line.
Its only purpose is to demonstrate one of the scalability ideas.
"""
import sys
import time
for line in sys.stdin:
    time.sleep(0.001)

Overwriting demo/mapper.py


In [158]:
# Make sure the mapper is executable
!chmod a+x demo/mapper.py

__Run the next three cells to apply our demo mapper with 1, 2 and 4 partitions.__

In [159]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'demo/mapper.py'

4.38 s ± 23.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [160]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'demo/mapper.py'

2.3 s ± 18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [161]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'demo/mapper.py'

1.26 s ± 3.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


__Run the following cells to repeat this process with your word count algorithm.__

In [162]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

269 ms ± 4.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [163]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

272 ms ± 8.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [164]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

271 ms ± 4.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [165]:
%%timeit
!./pWordCount_v2.sh 8 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

309 ms ± 4.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [166]:
%%timeit
!./pWordCount_v2.sh 16 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

379 ms ± 7.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [167]:
%%timeit
!./pWordCount_v2.sh 32 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

523 ms ± 4.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Question 9: Embarrassingly Parallel
`"If any one of them can explain it," said Alice, (she had grown so large in the last few minutes that she wasn’t a bit afraid of interrupting him,) "I’ll give him sixpence. I don’t believe there’s an atom of meaning in it."`
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 12  </div>

### Q9 Tasks:

* __a) short response:__ Describe what we mean by 'Embarrassingly Parallel' in terms of word counting. Does this term describe a 'task'? an 'implementation of a task'? 

* __b) OPTIONAL__: Define this concept in terms of 'associative' and 'commutative' operations. [`HINT:` _Refer to Chapter 2 in DITP_ ]

### Q9 Student Answers:

> __a)__ Remember that embarrassing parallel applies to computation capable of being divided into completely independent parts, each of which can execute on a separate processor as a self-contained process.  In the context of word counting, an EP framework describes the ability to chunk the input file, then run mappers, sort and shuffle, and finally utilize multiple reducers in parallel to produce output files of word-count key-value pairs.  EP is an implementation of a task.  It is not enough to run a job in parallel (a task); a framework must exist to exploit the benefits of end-to-end parallel computation. 

> __b)__ To consider computation as EP, the values of key-value pairs must possess associative and commutative properties.  

> The **associative property** describes an expression in which the sequence of operations are performed does not affect the outcome as long as the sequence of the operands is not changed.  For example:  

> $(2 + 3) + 4 = 2 + (3 +4) = 9$  or $2 x (3 x 4) = (2 x 3) x 4 = 24$  

> The **commutative property** describes an expression in which changing the order of the operands does not alter the result. For example:  

> $10 x 5 = 5 x 10 = 50$


### Congratulations, you have completed HW1! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/e/1FAIpQLSce9feiQeSkdP43A0ZYui1tMGIBfLfzb0rmgToQeZD9bXXX8Q/viewform